In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터수집2_동적</font></b>
# 1절. Selenium을 이용한 동적 웹크롤링 문법
Selenium docs : https://selenium-python.readthedocs.io/
`pip install sleninum`

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [3]:
driver = webdriver.Chrome()
driver.get("http://www.python.org")
elem = driver.find_element(By.NAME, 'q')
elem.clear()
elem.send_keys('pycon')

In [4]:
from selenium.webdriver.common.keys import Keys
elem.send_keys(Keys.RETURN)  

In [5]:
btn_elem = driver.find_element(By.CSS_SELECTOR, 'button#submit')
btn_elem.click()

In [6]:
result_list = driver.find_elements(By.CSS_SELECTOR, 'li > h3 > a')
for result in result_list : 
    title = result.text
    link  = result.get_attribute('href')
    print('{} - {}'.format(title, link))

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, result.attrs['href']))

PSF PyCon Trademark Usage Policy - /psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - /events/python-events/378/
PyCon Australia 2013 - /events/python-events/57/
PyCon AU 2019 - /events/python-events/776/
PyCon NL 2025 - /events/python-events/2084/
PyCon Australia 2014 - /events/python-events/10/
PyCon Ireland 2012 - /events/python-events/76/
PyCon Ireland 2016 - /events/python-events/429/
PyCon Ireland 2022 - /events/python-events/1320/
PyCon Australia 2014 - /events/python-events/1447/
PyCon Ireland 2024 - /events/python-events/1862/
PyCon APAC 2025 - /events/python-events/1879/
PyCon AU 2018 - /events/python-events/696/
PyCon APAC 2022 - /events/python-events/1216/
PyCon PH 2024 - /events/python-events/1661/
PyCon Ireland 2023 - /events/python-events/1568/
PyCon PL 2014 - /events/python-events/191/
PyCon MY 2015 - /events/python-events/313/
PyCon Ireland 2015 - /events/python-events/333/
PyCon AU 2015 - /events/python-events/273/


In [8]:
from urllib.parse import urlparse
current_url = driver.current_url # 셀레니윰을 통해 접근한 url
print(current_url)
parse_url = urlparse(current_url)
print(parse_url)
domain = f'{parse_url.scheme}://{parse_url.netloc}'
print(domain)

https://www.python.org/search/?q=pycon&submit=
ParseResult(scheme='https', netloc='www.python.org', path='/search/', params='', query='q=pycon&submit=', fragment='')
https://www.python.org


In [9]:
result_list = soup.select('li > h3 > a')
for result in result_list:
    print("{} - {}".format(result.text, domain+result.attrs['href']))

PSF PyCon Trademark Usage Policy - https://www.python.org/psf/trademarks/pycon
PyCon Italia 2016 (PyCon Sette) - https://www.python.org/events/python-events/378/
PyCon Australia 2013 - https://www.python.org/events/python-events/57/
PyCon AU 2019 - https://www.python.org/events/python-events/776/
PyCon NL 2025 - https://www.python.org/events/python-events/2084/
PyCon Australia 2014 - https://www.python.org/events/python-events/10/
PyCon Ireland 2012 - https://www.python.org/events/python-events/76/
PyCon Ireland 2016 - https://www.python.org/events/python-events/429/
PyCon Ireland 2022 - https://www.python.org/events/python-events/1320/
PyCon Australia 2014 - https://www.python.org/events/python-events/1447/
PyCon Ireland 2024 - https://www.python.org/events/python-events/1862/
PyCon APAC 2025 - https://www.python.org/events/python-events/1879/
PyCon AU 2018 - https://www.python.org/events/python-events/696/
PyCon APAC 2022 - https://www.python.org/events/python-events/1216/
PyCon PH 2

In [10]:
driver.close() # 브라우저 종료

# 2절. 동적웹크롤링 예제
## 2.1 다음뉴스검색

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('http://www.daum.net/')
# driver.implicitly_wait(0.5) # 0.5초 동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()

검색할 단어는?비크코인


In [12]:
# 뉴스 탭 클릭
#driver.find_elements(By.CSS_SELECTOR, 'ul.list_tab > li')[1].click()
driver.find_element(By.LINK_TEXT, '뉴스').click()

In [13]:
# 뉴스 제목과 link를 list 추가
news_list = []
strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
print(len(strongs))
for strongs in strongs:
    a = strongs.find_element(By.TAG_NAME, 'a')
    title = a.text
    link = a.get_attribute('href')
    news_list.append([title, link])

10


In [14]:
page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
print(page_nav.text)
next_page = page_nav.find_element(By.LINK_TEXT, "2")
next_page.click()

1 2 3 4 5


In [15]:
driver.close()

## 2.2 페이징 처리
- 다음 뉴스 페이징 처리 : 원하는 keyword를 원하는 페이지만큼 추출

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
driver = webdriver.Chrome()
driver.get('http://www.daum.net/')
# driver.implicitly_wait(0.5) # 0.5초 동안 대기
time.sleep(0.5) # 0.5초 멈춤
driver.find_element(By.CLASS_NAME, 'tf_keyword').click()
query = input('검색할 단어는?')
driver.find_element(By.CSS_SELECTOR, 'input[type="text"]').send_keys(query)
driver.find_element(By.CLASS_NAME, 'btn_ksearch').click()
time.sleep(2)
# 뉴스탭 이동
driver.find_element(By.LINK_TEXT, '뉴스').click()

# 뉴스 제목과 link를 list 추가
news_list = []
pages = 3 # int(input('몇 페이지 크롤링할까'))
for page in range(1, pages+1):
    strongs = driver.find_elements(By.CSS_SELECTOR, 'div.item-title > strong')
    # print(len(strongs))
    for strong in strongs:
        a = strong.find_element(By.TAG_NAME, 'a')
        title = a.text
        link = a.get_attribute('href')
        news_list.append([title, link])
    page_nav = driver.find_element(By.CSS_SELECTOR, 'div.inner_paging')
    next_page = page_nav.find_element(By.LINK_TEXT, str(page+1))
    next_page.click()
    time.sleep(2)
driver.close()
import pandas as pd
pd.DataFrame(news_list, columns=['title', 'link'])

검색할 단어는?비트코인


,title,link
0,[ET단상] 뒷전 된 대국민 투자기회…5년 늦은 비트코인 현물 ETF,http://v.daum.net/v/20251112160328227
1,가상자산 일제히 급락… 비트코인 10만달러 위태,http://v.daum.net/v/20251112080939100
2,중국 '가상화폐 여왕' 털자 비트코인 9조 '와르르'...도망간 영국서 실형,http://v.daum.net/v/20251112100711544
3,큰손들은 털고 나간다…비트코인 10만 달러 붕괴 조짐,http://v.daum.net/v/20251112082149370
4,금값 반등하자 다시 흔들리는 비트코인 [매일코인],http://v.daum.net/v/20251112164203997
5,"""연말에는 오를까""…비트코인, ETF 자금 유입 저조에 1억5400만원대 후퇴",http://v.daum.net/v/20251112095516978
6,"""최대 12배 간다""…국내 인기코인 리플, 이르면 내일 현물 ETF 출시",http://v.daum.net/v/20251112153215561
7,비트코인 9조 4천억 압수...자금세탁 중국인 징역11년8개월,http://v.daum.net/v/20251112073602477
8,"""9조 원 비트코인 압수""… 中 '가상화폐 여왕', 영국서 징역 11년",http://v.daum.net/v/20251112171216319
9,"9조원대 비트코인 보유한 사기꾼 징역 11년 8월형, 체포 순간 보니…",http://v.daum.net/v/20251112163415681


## 2.3 맞춤법 검사기
- 네이버 맞춤법 검사기 이용

In [17]:
from bs4 import BeautifulSoup

In [18]:

driver = webdriver.Chrome()
driver.get('https://www.naver.com/')
time.sleep(1)
elem = driver.find_element(By.ID, 'query')
elem.send_keys('맞춤법 검사기')
elem.send_keys(Keys.RETURN) # form태그의 submit 클릭 효과

textarea = driver.find_element(By.CLASS_NAME, 'txt_gray')
textarea.clear()
textarea.send_keys('안뇽하세요. 반갑숩니다. 감샤합니다')
btn = driver.find_element(By.CLASS_NAME, 'btn_check')
btn.click()
time.sleep(2)

soup = BeautifulSoup(driver.page_source, 'html.parser')
result = soup.select_one('p._result_text.stand_txt').text
print(result)
driver.close()

안녕하세요. 반갑습니다. 감사합니다


- 맞춤법검사전.txt 읽어 300자 단위로 짤라 list에 넣어 동적웹크롤링을 한후, 결과 출력

In [ ]:
with open('data/ch14_맞춤법검사전.txt', 'r', encoding='utf-8') as f:
    text = f.read()
ready_text = []
while(len(text) > 300):
    temp = text[:300]
    last_dot_index = temp.rfind('\n')
    ready_text.append(text[:last_dot_index])
    text = text[last_dot_index+1 : ]
ready_text.append(text)
print([len(ready) for ready in ready_text])
for ready in ready_text:
    print(len(ready))
